In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
ABtest = pd.read_csv('E:/DATA/ABtest/ab_data.csv')

In [4]:
ABtest.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [5]:
ABtest.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [6]:
t_old_error=len(ABtest[(ABtest['group']=='treatment')&(ABtest['landing_page'] == 'old_page')])
c_new_error = len(ABtest[(ABtest['group'] == 'control') & (ABtest['landing_page'] == 'new_page')])
print('实验组看到老页面人数：{},对照组看到新页面人数：{}'.format(t_old_error,c_new_error))

实验组看到老页面人数：1965,对照组看到新页面人数：1928


In [7]:
ABtest = ABtest[((ABtest['group'] == 'treatment') & (ABtest['landing_page'] == 'new_page'))
                      | ((ABtest['group'] == 'control') & (ABtest['landing_page'] == 'old_page'))]
print('正常数据',len(ABtest))

正常数据 290585


In [8]:
ABtest.drop_duplicates(subset='user_id').head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [9]:
ABtest.drop_duplicates(subset=['user_id'],inplace=True,keep='first')
ABtest=ABtest.reset_index()
ABtest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290584 entries, 0 to 290583
Data columns (total 6 columns):
index           290584 non-null int64
user_id         290584 non-null int64
timestamp       290584 non-null object
group           290584 non-null object
landing_page    290584 non-null object
converted       290584 non-null int64
dtypes: int64(3), object(3)
memory usage: 13.3+ MB


In [10]:
zb=ABtest[ABtest.landing_page=="new_page"].shape[0]/ABtest.shape[0]
print('看到新页面的用户占比：','%.2f%%'%(zb*100))

看到新页面的用户占比： 50.01%


In [14]:
n_old = ABtest.query('group=="control"').shape[0]
n_new = ABtest.query('group=="treatment"').shape[0]
convert_old = ABtest.query('group=="control" & converted==1').shape[0]
convert_new = ABtest.query('group=="treatment" & converted==1').shape[0]
p_old = convert_old / n_old
p_new = convert_new / n_new
p_c = (convert_old + convert_new)/(n_old + n_new)
z = (p_old - p_new)/ np.sqrt(p_c*(1 - p_c)*( 1/n_old + 1/n_new))
from scipy.stats import norm
z_alpha = norm.ppf(0.05)
print('旧版总受试用户数:', n_old, '旧版转化用户数:', convert_old, '旧版转化率:', p_old)
print('新版总受试用户数:', n_new, '新版转化用户数:', convert_new, '新版转化率:', p_new)
print('转化率的联合估计:', p_c)
print('检验统计量z:', z)
print(z_alpha)
result = "落入拒绝域，拒绝零假设" if abs(z) > abs(z_alpha) else "接受零假设"
print(result)

旧版总受试用户数: 145274 旧版转化用户数: 17489 旧版转化率: 0.1203863045004612
新版总受试用户数: 145310 新版转化用户数: 17264 新版转化率: 0.11880806551510564
转化率的联合估计: 0.11959708724499628
检验统计量z: 1.3109241984234394
-1.6448536269514729
接受零假设
